In [11]:
from eval import evaluate_single_input
from datasets import load_dataset
import faiss
import numpy as np
import pickle
import openai
from openai import OpenAI
!pip install rank_bm25
from rank_bm25 import BM25Okapi  # Sparse retrieval (BM25)

In [12]:
# Initialize DeepInfra OpenAI Client
openai = OpenAI(
    api_key="2Q2AU9IG4jKLdqmRrHc2UxaLP8hHB0ii",  # Replace with your key
    base_url="https://api.deepinfra.com/v1/openai",
)

In [13]:
def load_faiss_index(index_path, metadata_path):
    """
    Loads the FAISS index and metadata from local storage.
    """
    index = faiss.read_index(index_path)
    with open(metadata_path, 'rb') as f:
        metadata = pickle.load(f)
    return index, metadata

In [14]:
def search_faiss(query, top_k, index_path, metadata_path):
    """
    Searches FAISS for the most relevant chunks to the query.
    """
    # Load index and metadata
    index, metadata = load_faiss_index(index_path, metadata_path)

    # Generate embedding for the query
    response = openai.embeddings.create(
        model="BAAI/bge-m3",
        input=query,
        encoding_format="float"
    )
    query_embedding = np.array([response.data[0].embedding])

    # Search FAISS index for the top_k results
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve matching chunks
    results = []
    for i, idx in enumerate(indices[0]):
        results.append((metadata[idx], distances[0][i]))
    return results

In [15]:
# Load the dataset
dataset = load_dataset(
path='BlackFear/istanbul-qa-dataset',
trust_remote_code=True,
)

data = dataset['test']

# Extract data
queries = data['question']
references = data['reference']


In [16]:
def generate_answer(query, context):
    """
    Generates an answer based on the query and retrieved context using GPT.
    """
    # Prepare the input prompt
    prompt = f"""
    Use the following contexts to answer the question:

    Context:
    {context}

    Question:
    {query}

    Answer:
    """

    # Query the Llama model
    response = openai.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[
            {"role": "system", "content": "You will be answering questions about Istanbul. Please provide the answer to the following question."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

In [17]:
def hyde(query):
    """
    Generates a hypothetical document based on the query using LLM (Query2Doc).
    Combines the query and hypothetical document for retrieval.
    """
    # Prompt LLM to create a hypothetical document
    prompt = f"""
    Generate a detailed hypothetical document based on the query below:
    Just give the document nothing else.
    Query: {query}

    Hypothetical Document:
    """
    response = openai.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{'role': 'user', 'content': prompt}],
    )
    hypothetical_doc = response.choices[0].message.content.strip()

    # Combine query with the hypothetical document
    return hypothetical_doc

In [18]:
def hybrid_search(query, top_k, index_path, metadata_path):
    """
    Combines dense (FAISS) and sparse (BM25) retrieval for better results.
    """
    # Load FAISS index and metadata
    index, metadata = load_faiss_index(index_path, metadata_path)

    # Dense Retrieval (FAISS)
    # Generate embedding for query
    response = openai.embeddings.create(
        model="BAAI/bge-m3",  # Dense embedding model
        input=query,
        encoding_format="float"
    )
    query_embedding = np.array([response.data[0].embedding])
    distances, indices = index.search(query_embedding, top_k)

    # Sparse Retrieval (BM25)
    tokenized_metadata = [doc.split() for doc in metadata]  # Preprocess metadata
    bm25 = BM25Okapi(tokenized_metadata)
    sparse_scores = bm25.get_scores(query.split())

    # Normalize and combine scores (Hybrid Search)
    dense_scores = 1 / (1 + distances[0])  # Convert FAISS distances to similarity
    sparse_scores = np.array(sparse_scores)
    sparse_scores = sparse_scores / np.max(sparse_scores)  # Normalize BM25 scores

    # Combine dense and sparse scores (50-50 weight)
    combined_scores = 0.5 * dense_scores + 0.5 * sparse_scores[indices[0]]

    # Sort results based on combined scores
    sorted_indices = np.argsort(-combined_scores)  # Descending order
    results = [(metadata[indices[0][i]], combined_scores[i]) for i in sorted_indices]

    return results[:top_k]

In [19]:
def rag_pipeline(query, top_k=3, method="hyde",retrive_method='hybrid', index_path='index.faiss', metadata_path='metadata.pkl'):
    """
    Executes the RAG pipeline with different retrieval methods.
    Options: 'query2doc', 'hyde', 'hybrid'
    """
    # Modify query based on retrieval method
    if method == 'query2doc':
        query = query + " " + hyde(query)  # Query2Doc method
    elif method == 'hyde':
        query = hyde(query)  # Hypothetical document only for retrieval

    # Perform retrieval based on method
    if retrive_method == 'hybrid':
        results = hybrid_search(query, top_k, index_path, metadata_path)
    else:
        results = search_faiss(query, top_k, index_path, metadata_path)

    # Combine retrieved chunks as context
    context = "\n\n".join([f"Chunk {i+1}: {result[0]}" for i, result in enumerate(results)])

    # Generate an answer using GPT
    answer = generate_answer(query, context)

    # Return the answer and retrieved results
    return answer, results

In [20]:
from tqdm import tqdm
scores = []
for i in tqdm(range(len(queries)), desc="Processing queries", total=len(queries)):
    query = queries[i]
    reference = references[i]
    answer,results = rag_pipeline(query, top_k=3, method="",retrive_method='hybrid', index_path='faiss_index_recursive.bin', metadata_path='metadata_recursive.pkl')
    results = [result[0] for result in results]
    score = evaluate_single_input(query, reference, answer, results)
    print(score)
    scores.append(score)


Processing queries:   0%|          | 1/300 [00:04<20:03,  4.03s/it]

{'Faithfulness': 0.9876, 'Relevancy': 0.7, 'Factual Correctness': 0.75, 'BLEU Score': 0.14088803229434801, 'Overall Score': 0.80628}


Processing queries:   1%|          | 2/300 [00:11<29:49,  6.00s/it]

{'Faithfulness': 0.0, 'Relevancy': 1.0, 'Factual Correctness': 1.0, 'BLEU Score': 0.05873949094699214, 'Overall Score': 0.7}


Processing queries:   1%|          | 3/300 [00:14<23:35,  4.77s/it]

...

{'Faithfulness': 0.8976, 'Relevancy': 0.8, 'Factual Correctness': 0.9876, 'BLEU Score': 0.40185699044723144, 'Overall Score': 0.90432}


Processing queries: 100%|█████████▉| 299/300 [20:26<00:03,  3.82s/it]

{'Faithfulness': 1.0, 'Relevancy': 0.8, 'Factual Correctness': 0.5, 'BLEU Score': 0.42436787934785464, 'Overall Score': 0.74}


Processing queries: 100%|██████████| 300/300 [20:30<00:00,  4.10s/it]

{'Faithfulness': 1.0, 'Relevancy': 0.8, 'Factual Correctness': 0.9876, 'BLEU Score': 0.23770841795864292, 'Overall Score': 0.9350400000000001}


In [21]:
faithfullness, relevance, factual, bleu, overall = 0, 0, 0, 0, 0
for i, score in enumerate(scores):
    faithfullness += scores[i]['Faithfulness']
    relevance += scores[i]['Relevancy']
    factual += scores[i]['Factual Correctness']
    bleu += scores[i]['BLEU Score']
    overall += scores[i]['Overall Score']

faithfullness /= len(scores)
relevance /= len(scores)
factual /= len(scores)
bleu /= len(scores)
overall /= len(scores)

In [22]:
print(f"Faithfullness: {faithfullness}")
print(f"Relevance: {relevance}")
print(f"Factual Correctness: {factual}")
print(f"BLEU Score: {bleu}")
print(f"Overall Score: {overall}")
"""Faithfullness: 0.6501929999999998
Relevance: 0.6873533333333337
Factual Correctness: 0.5776309999999997
BLEU Score: 0.18773455804204672
Overall Score: 0.6323162999999997"""

Faithfullness: 0.6501929999999998
Relevance: 0.6873533333333337
Factual Correctness: 0.5776309999999997
BLEU Score: 0.18773455804204672
Overall Score: 0.6323162999999997
